In [1]:
import requests, json
from bs4 import BeautifulSoup

In [2]:
def food_info(name):
    '''
    This function gives you food information for the given input.

    PARAMETERS
        - name(str): name of Korean food in Korean ex) food_info("김치찌개")
    RETURN
        - res(list): list of dict that containing info for some Korean food related to 'name'
            - res['name'](str): name of food
            - res['ingredients'](str): ingredients to make the food
            - res['recipe'](list[str]): contain recipe in order
    '''
    url = f"https://www.10000recipe.com/recipe/list.html?q={name}"
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
    else : 
        print("HTTP response error :", response.status_code)
        return
    
    food_list = soup.find_all(attrs={'class':'common_sp_link'})
    food_id = food_list[0]['href'].split('/')[-1]
    new_url = f'https://www.10000recipe.com/recipe/{food_id}'
    new_response = requests.get(new_url)
    if new_response.status_code == 200:
        html = new_response.text
        soup = BeautifulSoup(html, 'html.parser')
    else : 
        print("HTTP response error :", response.status_code)
        return
    
    food_info = soup.find(attrs={'type':'application/ld+json'})
    result = json.loads(food_info.text)
    ingredient = ','.join(result['recipeIngredient'])
    recipe = [result['recipeInstructions'][i]['text'] for i in range(len(result['recipeInstructions']))]
    for i in range(len(recipe)):
        recipe[i] = f'{i+1}. ' + recipe[i]
    
    res = {
        'name': name,
        'ingredients': ingredient,
        'recipe': recipe
    }

    return res

In [5]:
food_info("김치전")

{'name': '김치전',
 'ingredients': '김치 2컵,부침가루 1컵,김치국물 1/3컵,물 2/3컵,다진마늘 1/2스푼',
 'recipe': ['1. 먼저 김치는 볼에 넣어 가위를 이용해 쫑쫑 잘게 썰어서 준비합니다 \u200b 방송에서는 줄기부분이 아삭해서 줄기부분 위주로 쓴다고 하는데 저는 그냥 섞어서 사용했어요',
  '2. 그리고 부침가루 1컵 가득 넣고 김치국물 1/3컵 정도 넣어주고 다진마늘 반스푼도 넣어줍니다 \u200b 수미네반찬 포인트는 바로 다진마늘! 다진마늘 넣으니까 뭔가 감칠맛이 더 나는것 같았어요',
  '3. 물도 2/3컵 넣어주는데요 물은 넣어가며 반죽의 농도를 맞춰주면 되요 반죽이 너무 되다면 물을 살짝 추가해줍니다 물 넣고 쉐킷쉐킷 잘 섞어주세요',
  '4. 기름두른 팬에 반죽을 올려 동그랗게 모양을 잡아줘 앞뒤로 노릇하게 구워줍니다',
  '5. 접시에 예쁘게 담아주니 먹음직스러운 김치전 만들기 완성♥ \u200b']}

In [6]:
import pandas as pd 

# Create an empty list to store the results
food_data = []

# List of food names to get information for
food_names = ["김치전", "비빔밥", "불고기", ""]

# Loop through each food name, get the information and append to the list
for name in food_names:
  info = food_info(name)
  if info:
    food_data.append(info)

# Convert the list of dictionaries to a DataFrame
df_food = pd.DataFrame(food_data)

# Display the DataFrame
df_food

,name,ingredients,recipe
0,김치전,"김치 2컵,부침가루 1컵,김치국물 1/3컵,물 2/3컵,다진마늘 1/2스푼",[1. 먼저 김치는 볼에 넣어 가위를 이용해 쫑쫑 잘게 썰어서 준비합니다 ​ 방송에...
1,비빔밥,"돼지고기 100g,애호박 1/5개,양파 1/2개,당근 1/3개,고추장 2T,간장 2...","[1. 양파, 당근, 애호박을 채썰어서 준비합니다., 2. 비빔밥에 비벼먹을 양념..."
2,불고기,"소불고기 400g,식용유 약간,물 1/4cup,후추 약간,송송 썬 쪽파 1대,간장 ...","[1. 불고기는 먹기 좋은 크기로 썬다., 2. 양념장을 만든다., 3. 불고기에 ..."
